In [1]:
# import necessary modules
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from classy import Class
import sys 
import ipywidgets

# LCDM parameters
common_settings = {'output':'tCl,pCl,lCl,mPk',
                   'lensing':'yes',
                   # fixed LambdaCDM parameters
                   'omega_b':0.022032,
                   'omega_cdm':0.12038,
                   'h':0.67556,
                   'A_s':2.215e-9,
                   'n_s':0.9619,
                   'tau_reio':0.0925,
                   # other output and precision parameters
                   'P_k_max_1/Mpc':3.0,
                   'longrangescalar':'n',
                   'longrangescalar_pt':'n'}  

# Matplotlib options
plt.rcParams["font.serif"] = ['Computer Modern']
plt.rcParams["text.usetex"] = True
plt.rcParams["lines.linewidth"] = 3
plt.rcParams["axes.linewidth"] = 1.
plt.rcParams["font.size"] = 15.
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.linestyle"] = "dotted"
plt.rcParams["axes.xmargin"] = 0
plt.rcParams["xtick.top"] = True
plt.rcParams["ytick.right"] = True
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["ytick.direction"] = "in"
plt.rcParams["xtick.minor.visible"] = True
plt.rcParams["ytick.minor.visible"] = True
plt.rcParams["figure.figsize"] = [14.0, 6.0]

We first compute $\Lambda$CDM, with and without neutrino masses

In [2]:
# Lambda CDM
LCDM = Class()
LCDM.set(common_settings)
LCDM.compute()

CMB_LCDM = LCDM.lensed_cl(2500)
ll_LCDM = CMB_LCDM['ell'][2:]
clTT_LCDM = CMB_LCDM['tt'][2:]

h = LCDM.h() # get reduced Hubble for conversions to 1/Mpc
kk = np.logspace(-4,np.log10(3),1000)
Pk_LCDM = np.array([LCDM.pk(k*h, 0.)*h**3 for k in kk]) # P(k) in (Mpc/h)**3 at zero redshift

# Lambda CDM with massive neutrinos at current Planck limit
LCDM_mnu = Class()
LCDM_mnu.set(common_settings)
LCDM_mnu.set({'N_ur': 0, # No relativistic species
          'N_ncdm': 3, # Three massive neutrinos
          'm_ncdm': "0.033, 0.033, 0.033", # Sum of masses of 0.1 eV
          'T_ncdm': "0.716486, 0.716486, 0.716486" # N_eff=3.046
         })
LCDM_mnu.compute()

CMB_LCDM_mnu = LCDM_mnu.lensed_cl(2500)
ll_LCDM_mnu = CMB_LCDM_mnu['ell'][2:]
clTT_LCDM_mnu = CMB_LCDM_mnu['tt'][2:]

h = LCDM_mnu.h() # get reduced Hubble for conversions to 1/Mpc
Pk_LCDM_mnu = np.array([LCDM_mnu.pk(k*h, 0.)*h**3 for k in kk]) # P(k) in (Mpc/h)**3 at zero redshift

And now our model

In [4]:
# Our modification
ll_lrs = {}
clTT_lrs = {}
Pk_lrs = {}

lrs = Class()
lrs.set(common_settings)
lrs.set({'N_ur':0}) # Remove massless neutrinos

for m_nu in [0.05, 0.1, 1]:
    print("Computing m_nu = %.2f eV" % m_nu)
    sys.stdout.flush()
    for g_mnu_over_M in np.geomspace(1e-1, 1e7, 9):
        print("\tComputing g m_nu/M = %.0e" % g_mnu_over_M)
        sys.stdout.flush()
        lrs.set({'longrangescalar':'y',
                 'longrangescalar_pt':'n',
                 'longrangescalar_nuggets':'y',
                 'log10_lrs':'n',
                 'lrs_M_phi': 1e-10,
                 'lrs_m_F': m_nu, # Mass of the sourcing fermion [eV]
                 'lrs_g_over_M': g_mnu_over_M/m_nu, # Coupling divided by the scalar mass [eV^-1]
                 'lrs_g_F': 6, # Number of fermionic degrees of freedom
                 'lrs_T_F': 0.716486 # Ratio among the fermionic and photon temperatures. This is (4/11)^1/3 * (N_eff/3)^1/4})
                })
    
        lrs.compute()

        CMB_lrs = lrs.lensed_cl(2500)
        ll_lrs[(m_nu, g_mnu_over_M)] = CMB_lrs['ell'][2:]
        clTT_lrs[(m_nu, g_mnu_over_M)] = CMB_lrs['tt'][2:]

        h = lrs.h() # get reduced Hubble for conversions to 1/Mpc
        Pk_lrs[(m_nu, g_mnu_over_M)] = np.array([lrs.pk(k*h, 0.)*h**3 for k in kk]) # P(k) in (Mpc/h)**3 at zero redshift

Computing m_nu = 0.05 eV
	Computing g m_nu/M = 1e-01
	Computing g m_nu/M = 1e+00
	Computing g m_nu/M = 1e+01
	Computing g m_nu/M = 1e+02
	Computing g m_nu/M = 1e+03
	Computing g m_nu/M = 1e+04
	Computing g m_nu/M = 1e+05
	Computing g m_nu/M = 1e+06
	Computing g m_nu/M = 1e+07
Computing m_nu = 0.10 eV
	Computing g m_nu/M = 1e-01
	Computing g m_nu/M = 1e+00
	Computing g m_nu/M = 1e+01
	Computing g m_nu/M = 1e+02
	Computing g m_nu/M = 1e+03
	Computing g m_nu/M = 1e+04
	Computing g m_nu/M = 1e+05
	Computing g m_nu/M = 1e+06
	Computing g m_nu/M = 1e+07
Computing m_nu = 1.00 eV
	Computing g m_nu/M = 1e-01
	Computing g m_nu/M = 1e+00
	Computing g m_nu/M = 1e+01
	Computing g m_nu/M = 1e+02
	Computing g m_nu/M = 1e+03
	Computing g m_nu/M = 1e+04
	Computing g m_nu/M = 1e+05
	Computing g m_nu/M = 1e+06
	Computing g m_nu/M = 1e+07


In [5]:
# Plot the CMB
plt.rcParams["figure.figsize"] = [28.0, 12.0]
plt.rcParams["font.size"] = 30.

def plot_CMB(m_nu, log10_g_mnu_over_M):
    g_mnu_over_M = 10**log10_g_mnu_over_M
    plt.subplot(1, 2, 1)
    plt.xscale("log")
    plt.xlabel(r'$\ell$')
    plt.ylabel(r'$[\ell(\ell+1)/2\pi]  C_\ell^\mathrm{TT}$')
    plt.plot(ll_LCDM,clTT_LCDM*ll_LCDM*(ll_LCDM+1)/2./np.pi, 
             label=r"$\Lambda$CDM", linewidth=2)
    plt.plot(ll_LCDM_mnu,clTT_LCDM_mnu*ll_LCDM_mnu*(ll_LCDM_mnu+1)/2./np.pi, 
             label=r"$\Lambda$CDM, $m_\nu=1$ eV", linewidth=2.5, linestyle="dashed")

    plt.plot(ll_lrs[(m_nu, g_mnu_over_M)],
             clTT_lrs[(m_nu, g_mnu_over_M)]*ll_lrs[(m_nu, g_mnu_over_M)]*(ll_lrs[(m_nu, g_mnu_over_M)]+1)/2./np.pi, 
             label=r"$m_\nu=%.2f$ eV, $\frac{g m_\nu}{M} = 10^%i$" % (m_nu, log10_g_mnu_over_M), linewidth=2)

    plt.legend()

    plt.subplot(1, 2, 2)
    plt.xscale("log")
    plt.xlabel(r'$\ell$')
    plt.plot(ll_LCDM,(clTT_LCDM - clTT_lrs[(m_nu, g_mnu_over_M)])/clTT_LCDM, 
             label=r"$\frac{{C_\ell^\mathrm{TT}}_{\Lambda\mathrm{CDM}} - {C_\ell^\mathrm{TT}}_\mathrm{mod}}{{C_\ell^\mathrm{TT}}_{\Lambda\mathrm{CDM}}}$",
             linewidth=2.5, color="C2")
    plt.plot(ll_LCDM,(clTT_LCDM - clTT_LCDM_mnu)/clTT_LCDM, 
             label=r"$\frac{{C_\ell^\mathrm{TT}}_{\Lambda\mathrm{CDM}} - {C_\ell^\mathrm{TT}}_\mathrm{\Lambda\mathrm{CDM}, m_\nu}}{{C_\ell^\mathrm{TT}}_{\Lambda\mathrm{CDM}}}$",
             linewidth=2.5, color="C1", linestyle="dashed")
    
ipywidgets.interact(plot_CMB, m_nu = [1, 0.1, 0.05], 
                    log10_g_mnu_over_M=ipywidgets.FloatSlider(min=-1, max=7, step=1, continuous_update=False))

interactive(children=(Dropdown(description='m_nu', options=(1, 0.1, 0.05), value=1), FloatSlider(value=0.0, co…

<function __main__.plot_CMB(m_nu, log10_g_mnu_over_M)>

In [27]:
# Plot LSS
plt.rcParams["figure.figsize"] = [28.0, 12.0]
plt.rcParams["font.size"] = 30.

def plot_LSS(m_nu, log10_g_mnu_over_M):
    g_mnu_over_M = 10**log10_g_mnu_over_M
    plt.subplot(1, 2, 1)
    plt.xscale("log")
    plt.yscale("log")
    plt.xlabel(r'$k \,\,\,\, [h/\mathrm{Mpc}]$')
    plt.ylabel(r'$P(k) \,\,\,\, [\mathrm{Mpc}/h]^3$')
    plt.plot(kk,Pk_LCDM, label=r"$\Lambda$CDM", linewidth=2.5)
    plt.plot(kk,Pk_LCDM_mnu, label=r"$\Lambda$CDM, $m_\nu=1$ eV", linestyle="dashed", linewidth=2)
    plt.plot(kk,Pk_lrs[(m_nu, g_mnu_over_M)],
             label=r"$m_\nu=%.2f$ eV, $\frac{g m_\nu}{M} = 10^%i$" % (m_nu, log10_g_mnu_over_M), linewidth=2)
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.xscale("log")
    plt.xlabel(r'$k \,\,\,\, [h/\mathrm{Mpc}]$')
    plt.plot(kk,(Pk_LCDM - Pk_lrs[(m_nu, g_mnu_over_M)])/Pk_LCDM, 
             label=r"$\frac{P(k)_{\Lambda\mathrm{CDM}} - P(k)_\mathrm{mod}}{P(k)_{\Lambda\mathrm{CDM}}}$",
             linewidth=2.5, color="C2")
    plt.plot(kk,(Pk_LCDM - Pk_LCDM_mnu)/Pk_LCDM, 
             label=r"$\frac{P(k)_{\Lambda\mathrm{CDM}} - P(k)_\mathrm{\Lambda\mathrm{CDM}, m_\nu}}{P(k)_{\Lambda\mathrm{CDM}}}$",
             linewidth=2.5, linestyle="dashed", color="C1")
    plt.legend()
    
ipywidgets.interact(plot_LSS, m_nu = [1, 0.1, 0.05], 
                    log10_g_mnu_over_M=ipywidgets.FloatSlider(min=-1, max=7, step=1, continuous_update=False))

interactive(children=(Dropdown(description='m_nu', options=(1, 0.1, 0.05), value=1), FloatSlider(value=0.0, co…

<function __main__.plot_LSS(m_nu, log10_g_mnu_over_M)>